In [ ]:
import sys
import argparse
import torch
import cv2
import time
import re
import numpy as np
from ultralytics import YOLO
import math
from typing import List
import os
import pandas as pd

model_path = '/home/agrotech/deep-gauge-master/Reading_Circular_Analogue_Gauges_using_Digital_Image_Processing/model_weights/pose_gauge.pt'
img_folder_path = "/path/to/your/images"  # Change this to your image folder path

def compute_angle(A, B, C):
    # Your existing compute_angle function
    # ...

def calculate_pointer_angle(img):
    model1 = YOLO(model_path)
    results = model1(img)
    theta1 = None  # Initialize with a default value
    theta2 = None  # Initialize with a default value

    for result in results:
        keypoints_set = result.keypoints.xy.cpu().numpy()
        print(keypoints_set)

        # Check if keypoints_set is empty
        if keypoints_set.size == 0:
            print('No Key Point Detected')
            theta1 = 0
            theta2 = 45
            break

        # Check if there are at least 4 keypoints in the set
        if keypoints_set.shape[1] < 4:
            print('Not enough keypoints detected')
            break

        print(keypoints_set[0])
        A, B, C, D = keypoints_set[0][0], keypoints_set[0][1], keypoints_set[0][3], keypoints_set[0][2]
        print(keypoints_set[0])
        print(f"A is {A[1]}")
        print(f"B is {B[1]}")
        print(f"C is {C[1]}")

        theta1 = compute_angle(A, B, C)
        theta2 = 360 - compute_angle(B, D, C)

        print(f"Angle between keypoints A, B, and C is: {theta1:.2f} degrees")
        print(f"Angle between keypoints B, C, and D is: {theta2:.2f} degrees")

    return theta1, theta2

def process_images(img_folder):
    result_data = {'File': [], 'Angle_ABC': [], 'Angle_BDC': []}

    for img_file in os.listdir(img_folder):
        img_path = os.path.join(img_folder, img_file)

        if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            frame = cv2.imread(img_path)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            angle_ABC, angle_BDC = calculate_pointer_angle(frame)

            result_data['File'].append(img_file)
            result_data['Angle_ABC'].append(angle_ABC)
            result_data['Angle_BDC'].append(angle_BDC)

    result_df = pd.DataFrame(result_data)
    result_df.to_excel('angles_results.xlsx', index=False)

if __name__ == "__main__":
    process_images(img_folder_path)
